In [71]:
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
import requests

In [14]:
# Scrapping for List of stocks in S&P500
url = input('Enter Wikipedia url for list of S&P500 Companies_')
  
wiki_data = pd.read_html(url)
print(wiki_data)

Enter Wikipedia url for list of S&P500 Companies_ https://en.wikipedia.org/wiki/List_of_S%26P_500_companies


[    Symbol              Security             GICS Sector  \
0      MMM                    3M             Industrials   
1      AOS           A. O. Smith             Industrials   
2      ABT                Abbott             Health Care   
3     ABBV                AbbVie             Health Care   
4      ACN             Accenture  Information Technology   
..     ...                   ...                     ...   
498    YUM           Yum! Brands  Consumer Discretionary   
499   ZBRA    Zebra Technologies  Information Technology   
500    ZBH         Zimmer Biomet             Health Care   
501   ZION  Zions Bancorporation              Financials   
502    ZTS                Zoetis             Health Care   

                      GICS Sub-Industry    Headquarters Location  Date added  \
0              Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1                     Building Products     Milwaukee, Wisconsin  2017-07-26   
2                 Health Care Equipmen

In [15]:
# Extracting Stock Price Historical Data from Yahoo! Finance using Tickers from Wikipedia
import yfinance as yf
from pandas.tseries.offsets import DateOffset
from datetime import datetime

ticker_df = wiki_data[0]
display(ticker_df.head(2))
display(ticker_df.tail(2))

def extract_stock_data():
    ticker_list= ticker_df['Symbol'].to_list()
    stock_list = ticker_df['Security'].to_list()
    print(stock_list)
    stock_name = input('Enter Stock_Name from the above displayed list for Analysis:')
    start_date = input('Enter Start Date for Historical Data in yyyy-mm-dd:')
    today = datetime.today().strftime('%Y-%m-%d')    
    row_index = ticker_df.query(f"Security == '{stock_name}'").index[0]
    stock_ticker = ticker_list[row_index]
    stock_df = yf.download(stock_ticker,start_date,today)
    stock_df.index = pd.to_datetime(stock_df.index)
    print(f"{stock_ticker}:{stock_name}")
    return(stock_df)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873
502,ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952


In [101]:
# Calling Specific Company stock data 
# Select Stock & Historical Date for Analysis
# using extract_stock_data() function

df = extract_stock_data()

display(df.head(3))
display(df.tail(3))

['3M', 'A. O. Smith', 'Abbott', 'AbbVie', 'Accenture', 'Adobe Inc.', 'Advanced Micro Devices', 'AES Corporation', 'Aflac', 'Agilent Technologies', 'Air Products and Chemicals', 'Airbnb', 'Akamai', 'Albemarle Corporation', 'Alexandria Real Estate Equities', 'Align Technology', 'Allegion', 'Alliant Energy', 'Allstate', 'Alphabet Inc. (Class A)', 'Alphabet Inc. (Class C)', 'Altria', 'Amazon', 'Amcor', 'Ameren', 'American Airlines Group', 'American Electric Power', 'American Express', 'American International Group', 'American Tower', 'American Water Works', 'Ameriprise Financial', 'Ametek', 'Amgen', 'Amphenol', 'Analog Devices', 'Ansys', 'Aon', 'APA Corporation', 'Apple Inc.', 'Applied Materials', 'Aptiv', 'Arch Capital Group', 'Archer-Daniels-Midland', 'Arista Networks', 'Arthur J. Gallagher & Co.', 'Assurant', 'AT&T', 'Atmos Energy', 'Autodesk', 'Automated Data Processing', 'AutoZone', 'AvalonBay Communities', 'Avery Dennison', 'Axon Enterprise', 'Baker Hughes', 'Ball Corporation', 'Bank

Enter Stock_Name from the above displayed list for Analysis: Fidelity National Information Services
Enter Start Date for Historical Data in yyyy-mm-dd: 2012-01-01


[*********************100%%**********************]  1 of 1 completed

FIS:Fidelity National Information Services


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-03,27.059999,27.110001,26.660000,26.690001,21.561054,1721700
2012-01-04,26.620001,26.780001,26.309999,26.700001,21.569132,1707000
2012-01-05,26.559999,27.200001,26.559999,26.900000,21.730694,1876000


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-03-14,68.540001,68.830002,67.739998,68.220001,68.220001,5345800
2024-03-15,67.580002,69.669998,67.510002,69.120003,69.120003,5771600
2024-03-18,69.089996,69.699997,68.470001,69.300003,69.300003,3051200


In [102]:
# Feature Engineering _ additionals features for ML Analysis

import pandas_ta as ta

def technical_indicators(OHLCV_df):
    # Daily Avg Price
    OHLCV_df['avg_price'] = OHLCV_df.ta.ohlc4()
    # Daily Returns
    OHLCV_df['daily_returns'] = OHLCV_df['Close'].pct_change()
    # 21Day Volatility
    OHLCV_df['volatility_21'] = OHLCV_df['Close'].rolling(window=21).std()
    # Simple Moving Avg 21Days & 50Days
    OHLCV_df['sma_21'] = OHLCV_df.ta.sma(length=21)
    OHLCV_df['sma_50'] = OHLCV_df.ta.sma(length=50)
    # Exponential Moving Avg 21Days & 50Days
    OHLCV_df['ema_21'] = OHLCV_df.ta.ema(length=21)
    OHLCV_df['ema_50'] = OHLCV_df.ta.ema(length=50)
    # VWAP modified to use Annual cummulative sum
    OHLCV_df['typical_price'] = (OHLCV_df['High']+OHLCV_df['Low']+OHLCV_df['Close'])/3
    OHLCV_df['vw_typical_price'] = OHLCV_df['typical_price']*OHLCV_df['Volume']
    OHLCV_df['vwap_annual'] = OHLCV_df.groupby(OHLCV_df.index.year)['vw_typical_price'].cumsum() / OHLCV_df.groupby(OHLCV_df.index.year)['Volume'].cumsum()
    # RSI Momemtum Indicator 14Days
    OHLCV_df['rsi_14'] = OHLCV_df.ta.rsi()
    # MACD Momentum Indicator
    OHLCV_df[['ema_12', 'ema_26', 'MACD_signal']] = OHLCV_df.ta.macd()
    # Bolinger Bands %B Indicator
    OHLCV_df[['BB_lower','BB_middle','BB_upper','BBP','BB%B']]= OHLCV_df.ta.bbands()
    # On-Balance Volume in Millions
    OHLCV_df['OBV_in_million'] = OHLCV_df.ta.obv()/1000000
    # Average Directional Index 14Days
    OHLCV_df[['adx_14','dmp_14','dmn_14']] = OHLCV_df.ta.adx()
    # Dropping trivial Columns
    OHLCV_append_df = OHLCV_df.drop(columns = ['typical_price','vw_typical_price',
                                               'BBP','dmp_14','dmn_14']).dropna()
    return(OHLCV_append_df)
   

In [103]:
# Extract Technical Indicators & Feature Engineering
# using technical_indicators function

df = technical_indicators(df)

df.tail()

,Open,High,Low,Close,Adj Close,Volume,avg_price,daily_returns,volatility_21,sma_21,...,rsi_14,ema_12,ema_26,MACD_signal,BB_lower,BB_middle,BB_upper,BB%B,OBV_in_million,adx_14
Date,,,,,,,,,,,,,,,,,,,,,
2024-03-12,70.019997,70.519997,69.529999,69.550003,69.550003,2722800,69.904999,-0.002867,3.094352,66.428572,...,71.441289,2.031388,0.154148,1.877239,69.111096,69.586002,70.060908,0.462099,349.3572,40.452921
2024-03-13,68.919998,70.029999,68.699997,68.900002,68.900002,4007800,69.137499,-0.009346,2.988864,66.741906,...,65.782549,1.916094,0.031084,1.885010,68.755021,69.400002,70.044982,0.112391,345.3494,40.414385
2024-03-14,68.540001,68.830002,67.739998,68.220001,68.220001,5345800,68.332500,-0.009869,2.722682,67.074287,...,60.393181,1.749684,-0.108261,1.857945,68.055755,69.214001,70.372248,0.070903,340.0036,39.235187
2024-03-15,67.580002,69.669998,67.510002,69.120003,69.120003,5771600,68.470001,0.013193,2.557171,67.382858,...,64.534614,1.671161,-0.149428,1.820588,68.034883,69.108002,70.181121,0.505592,345.7752,38.669177
2024-03-18,69.089996,69.699997,68.470001,69.300003,69.300003,3051200,69.139999,0.002604,2.439299,67.651906,...,65.315752,1.604954,-0.172507,1.777461,68.113046,69.018002,69.922959,0.655809,348.8264,38.161891


In [104]:
# Preparing Features & Target DataFrame for ML Analysis

def predictors_target(stock_df):
    # Previous Day's Data as predictors
    #predictors_df = stock_df.shift().dropna()
    #target_df = stock_df[['Close']].iloc[1:]
    predictors_df = stock_df
    target_df = stock_df[['Close']].shift(-1)
    return(predictors_df,target_df)

X,y = predictors_target(df)

display(X.tail())
display(y.tail())
    



,Open,High,Low,Close,Adj Close,Volume,avg_price,daily_returns,volatility_21,sma_21,...,rsi_14,ema_12,ema_26,MACD_signal,BB_lower,BB_middle,BB_upper,BB%B,OBV_in_million,adx_14
Date,,,,,,,,,,,,,,,,,,,,,
2024-03-12,70.019997,70.519997,69.529999,69.550003,69.550003,2722800,69.904999,-0.002867,3.094352,66.428572,...,71.441289,2.031388,0.154148,1.877239,69.111096,69.586002,70.060908,0.462099,349.3572,40.452921
2024-03-13,68.919998,70.029999,68.699997,68.900002,68.900002,4007800,69.137499,-0.009346,2.988864,66.741906,...,65.782549,1.916094,0.031084,1.885010,68.755021,69.400002,70.044982,0.112391,345.3494,40.414385
2024-03-14,68.540001,68.830002,67.739998,68.220001,68.220001,5345800,68.332500,-0.009869,2.722682,67.074287,...,60.393181,1.749684,-0.108261,1.857945,68.055755,69.214001,70.372248,0.070903,340.0036,39.235187
2024-03-15,67.580002,69.669998,67.510002,69.120003,69.120003,5771600,68.470001,0.013193,2.557171,67.382858,...,64.534614,1.671161,-0.149428,1.820588,68.034883,69.108002,70.181121,0.505592,345.7752,38.669177
2024-03-18,69.089996,69.699997,68.470001,69.300003,69.300003,3051200,69.139999,0.002604,2.439299,67.651906,...,65.315752,1.604954,-0.172507,1.777461,68.113046,69.018002,69.922959,0.655809,348.8264,38.161891


,Close
Date,
2024-03-12,68.900002
2024-03-13,68.220001
2024-03-14,69.120003
2024-03-15,69.300003
2024-03-18,NaN


In [105]:
# Data Normalizing & Train-Test Split

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

def train_test_split_scaled(predictors_df, target_df):
    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = X.iloc[21*33*i:(21*33*i)+(21*(33-test_range)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train = target_df.drop(index = predictors_test.index)
    target_test = target_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_test_scaled = predictors_scaled.transform(predictors_test)
    
    return(predictors_train_scaled,predictors_test_scaled,target_train,target_test)

X_train_scaled, X_test_scaled, y_train, y_test = train_test_split_scaled(X,y)


In [106]:
# Predicting Stock Price using Linear Models
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import MultiTaskElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge


def linear_models_prediction(predictors_train_scaled,predictors_test_scaled,
                             target_train,target_test):
    list_of_models = [Ridge(),Lasso(),MultiTaskElasticNet(),
                  LassoLars(),BayesianRidge()]
    target_predictions_df = target_test.copy()
    for model in list_of_models:
        model_instance = model
        model_instance.fit(predictors_train_scaled,target_train)
        model_prediction = model_instance.predict(predictors_test_scaled)
        target_predictions_df[f'{model}_Close'] = model_prediction

    return(target_predictions_df)

y_predictions = linear_models_prediction(X_train_scaled, X_test_scaled, y_train, y_test)

y_predictions
    

/Users/venkataaditya/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.567e+02, tolerance: 2.861e+02
  model = cd_fast.enet_coordinate_descent(
/Users/venkataaditya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Close,Ridge()_Close,Lasso()_Close,MultiTaskElasticNet()_Close,LassoLars()_Close,BayesianRidge()_Close
Date,,,,,,
2014-06-17,53.959999,54.211855,54.890105,55.704656,55.053770,54.189406
2014-06-18,54.200001,53.941526,54.889953,55.625448,54.942921,54.028277
2014-06-19,54.439999,54.320196,55.071607,55.848380,55.171330,54.271097
2014-06-20,54.599998,54.569266,55.384740,56.065920,55.401096,54.501148
2014-06-23,54.560001,54.647446,55.430808,56.169588,55.551231,54.622324
...,...,...,...,...,...,...
2024-03-12,68.900002,70.321591,70.359523,74.242897,70.028592,69.412226
2024-03-13,68.220001,69.455981,69.669017,73.662636,69.411390,68.794565
2024-03-14,69.120003,68.706313,68.871756,73.117649,68.762226,68.134619


In [107]:
# Plot Predictions using Linear Models

y_predictions.hvplot()



:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [ ]:
# Calculating Root_Mean_Square_Error(RMSE) 
# and Mean_Absolute_Percentage_Error(MAPE)